Wszystkie obliczenia wykonywane są z 256-bitową precyzją arytmetyki.

In [1]:
setprecision(256)

256

Biblioteka do rysowania wykresów

In [2]:
# using Pkg
# Pkg.add("Plots")
using Plots
plotly()

Plots.PlotlyBackend()

***
Funkcja obliczająca iloraz różnicowy z definicji 
$$f[x_0, x_1, ...,\, x_n] := \sum_{i=0}^n\frac{f(x_i)}
    {\prod\limits_{\substack{{j=0}\\{j\neq i}}}{(x_i - x_j)}}$$

In [3]:
function difference_quotient(points)
    xs = map(x -> x[1], points)
    ys = map(y -> y[2], points)
    n = size(xs)[1]

    res = BigFloat(0)
    
    for i in 1:n
        denominator = BigFloat(1)
        for j in 1:n
            if j != i
                denominator *= (xs[i] - xs[j])
            end
        end
        res += ys[i] / denominator
    end

    return res
end

difference_quotient (generic function with 1 method)

***
Funkcja obliczające kolejne przybliżenia miejsca zerowego funkcji $f(x)$ wykorzystująca metodę Müllera
$$x_{n+1} = x_n - 
\frac{2f(x_n)}
{\lambda \pm \sqrt{\lambda^2 -4f(x_n)f[x_n, x_{n-1}, x_{n-2}]}},$$

gdzie $$\lambda = f[x_n,x_{n-1}] + f[x_n, x_{n-2}] - f[x_{n-1}, x_{n-2}]$$

In [4]:
function mullers_method(x0, x1, x2, f)
    if x1 == x2
        return :end
    end
    
    # Często używane wartości funkcji są zapamiętywane 
    # aby uniknąć ich wielokrotnego obliczania
    f0 = f(x0)
    f1 = f(x1)
    f2 = f(x2)
    numerator = BigFloat(2.0) * f2

    λ = difference_quotient([(x2, f2), (x1, f1)]) +
        difference_quotient([(x2, f2), (x0, f0)]) -
        difference_quotient([(x1, f1), (x0, f0)])
    x0_x1_x2_diff_quot = difference_quotient([
        (x0, f0), 
        (x1, f1), 
        (x2, f2)
    ])

    # Obliczanie obu możliwych wartości mianownika
    denom_r = sqrt(λ^2.0 - 4.0*f(x2) * x0_x1_x2_diff_quot + 0im)
    denom_0 = λ - denom_r
    denom_1 = λ + denom_r

    # Wybór większej wartości mianownika
    denominator = abs(denom_0) > abs(denom_1) ? denom_0 : denom_1
    result = x2 - numerator/denominator
    return result
end

mullers_method (generic function with 1 method)

***
Funkcja obliczająca kolejne przybliżenie miejsca zerowego funkcji f(x) używająca metody Newtona
$$x_n := x_{n-1} - \frac{f(x_{n-1})}{f'(x_{n-1})}.\label{newtonformula}\tag{2}$$

In [5]:
function newtons_method(x0, f, df)
    return x0 - f(x0) / df(x0)
end

newtons_method (generic function with 1 method)

***
Funkcja obliczająca kolejne przybliżenie miejsca zerowego funkcji f(x) przy użyciu metody siecznych.
$$x_n:=x_{n-1} - f(x_{n-1})\frac{x_{n-1} - x_{n-2}}{f(x_{n-1}) - f(x_{n-2})}$$

In [6]:
function secant_method(x0, x1, f)
    if x1 == x0
        return :end
    end

    f0 = f(x0)
    f1 = f(x1)
    return x1 - f1 * (x1 - x0) / (f1 - f0)
end

secant_method (generic function with 1 method)

***
Funkcja obliczająca $x_1, x_2, ..., x_n$ dla wszystkich trzech metod.

In [7]:
function calculate_results(f, df, x0, n)
    x1 = newtons_method(x0, f, df)
    x2 = secant_method(x0, x1, f)
    
    newton_results = [x0]
    secant_results = [x0, x1]
    muller_results = Union{BigFloat, Complex{BigFloat}}[x0, x1, x2]

    newton_finished = false
    secant_finished = false
    muller_finished = false
    
    for i in 1:n
        if newton_finished && secant_finished && muller_finished
            break
        end
        
        if !newton_finished
            x_i = newtons_method(newton_results[i], f, df)
            if x_i == newton_results[i]
                newton_finished = true
            else
                push!(newton_results, x_i)
            end
        end
                
        if !secant_finished
            x_i = secant_method(
                secant_results[i],
                secant_results[i+1],
                f
            )
            if x_i == :end
                secant_finished = true
            else
                push!(secant_results, x_i)
            end
        end

        if !muller_finished
            x_i = mullers_method(
                muller_results[i],
                muller_results[i+1],
                muller_results[i+2],
                f
            )
            if x_i == :end
                muller_finished = true
            else
                push!(muller_results, x_i)
            end
        end
    end

    return newton_results, secant_results, muller_results
end

calculate_results (generic function with 1 method)

Funkcja wypisująca wyniki obliczeń dla wszystkich trzech metod

In [8]:
# Funkcja obliczająca liczbę poprawnych cyfr wyniku
function correct_digits(approx, accurate)
    delta = abs((accurate - approx) / accurate)
    return floor(-log10(delta))
end

function fill_array(array, n)
    if size(array)[1] > n
        throw("fill_array: Invalid input!")
    else
        while size(array)[1] < n
            push!(array, BigFloat(Inf))
        end
    end
end

function draw_plot(newton, secant, muller)
    n = max(size(newton)[1], size(secant)[1], size(muller)[1])
    fill_array(newton, n)
    fill_array(secant, n)
    fill_array(muller, n)
    plot(
        1:n,
        xticks=0:1:n,
        yticks=0:5:120,
        xlabel="\$n\$",
        ylabel="\$liczba\\, cyfr\$",
        label=["Newton" "sieczne" "Müller"],
        [newton, secant, muller],
        linewidth=2.0
    )
end

function test_methods(f, df, x0, zero, n)    
    newton0, secant0, muller0 = calculate_results(f, df, x0, n)
    newton, secant, muller = map(
        xs -> map(x -> correct_digits(x, zero), xs),
        (newton0, secant0, muller0)
    )
    
    function print_results(xs, name)
        println("Wyniki dla metody $name:\n")
        i = 1
        for x in xs
            istr = lpad(i-1, Int64(ceil(log10(n)))+1, '0')
            println("n=$istr --- $(xs[i])")
            i += 1
        end
        println("==========================================")
    end
    
    print_results(newton, "Newtona")
    print_results(secant, "siecznych")
    print_results(muller, "Müllera")

    draw_plot(newton, secant, muller)
end

test_methods (generic function with 1 method)

***
Testy dla różnych funkcji

In [9]:
f0(x) = x^2.0 - 3.0x + 2.0
df0(x) = 2.0x - 3.0
f0_zero = BigFloat(1.0)
x0_0 = BigFloat(-0.2)
@assert(f0(f0_zero) == 0.0)
test_methods(f0, df0, x0_0, f0_zero, 20)

Wyniki dla metody Newtona:

n=000 --- -1.0
n=001 --- 0.0
n=002 --- 1.0
n=003 --- 2.0
n=004 --- 4.0
n=005 --- 8.0
n=006 --- 1.6e+01
n=007 --- 3.3e+01
n=008 --- 6.7e+01
n=009 --- 7.6e+01
n=010 --- 7.6e+01
n=011 --- 7.6e+01
n=012 --- 7.6e+01
n=013 --- 7.6e+01
n=014 --- 7.6e+01
n=015 --- 7.6e+01
n=016 --- 7.6e+01
n=017 --- 7.6e+01
n=018 --- 7.6e+01
n=019 --- 7.6e+01
n=020 --- 7.6e+01
Wyniki dla metody siecznych:

n=000 --- -1.0
n=001 --- 0.0
n=002 --- 0.0
n=003 --- 1.0
n=004 --- 2.0
n=005 --- 3.0
n=006 --- 5.0
n=007 --- 8.0
n=008 --- 1.4e+01
n=009 --- 2.3e+01
n=010 --- 3.7e+01
n=011 --- 6.1e+01
n=012 --- 7.6e+01
n=013 --- 7.6e+01
n=014 --- Inf
n=015 --- Inf
Wyniki dla metody Müllera:

n=000 --- -1.0
n=001 --- 0.0
n=002 --- 0.0
n=003 --- 7.6e+01
n=004 --- 7.6e+01
n=005 --- Inf
n=006 --- Inf


In [10]:
f1(x) = sqrt(x)-4
df1(x) = 0.5*x^(-1.0/2.0)
f1_zero = 16.0
x0_1 = BigFloat(18.0)
@assert(f1(f1_zero) == 0.0, f1(f1_zero))
test_methods(f1, df1, x0_1, f1_zero, 20)

Wyniki dla metody Newtona:

n=000 --- 0.0
n=001 --- 2.0
n=002 --- 5.0
n=003 --- 1.1e+01
n=004 --- 2.3e+01
n=005 --- 4.7e+01
n=006 --- Inf
Wyniki dla metody siecznych:

n=000 --- 0.0
n=001 --- 2.0
n=002 --- 3.0
n=003 --- 6.0
n=004 --- 1.1e+01
n=005 --- 1.9e+01
n=006 --- 3.1e+01
n=007 --- 5.1e+01
n=008 --- Inf
n=009 --- Inf
Wyniki dla metody Müllera:

n=000 --- 0.0
n=001 --- 2.0
n=002 --- 3.0
n=003 --- 8.0
n=004 --- 1.5e+01
n=005 --- 2.8e+01
n=006 --- 5.3e+01
n=007 --- Inf
n=008 --- Inf


In [11]:
f2(x) = (x-1.0)*(x-2.0)*(2x-3.0)*(x-4.0)
df2(x) = 8.0(x^3.0 - 6.375x^2 + 12.25x - 7.25)
f2_zero = 2.0
x0_2 = BigFloat(3.0)
@assert(f2(f2_zero) == 0.0, f2(f2_zero))
test_methods(f2, df2, x0_2, f2_zero, 20)

Wyniki dla metody Newtona:

n=000 --- 0.0
n=001 --- 1.0
n=002 --- 1.0
n=003 --- 2.0
n=004 --- 5.0
n=005 --- 9.0
n=006 --- 1.9e+01
n=007 --- 3.7e+01
n=008 --- 7.4e+01
n=009 --- Inf
Wyniki dla metody siecznych:

n=000 --- 0.0
n=001 --- 1.0
n=002 --- 1.0
n=003 --- 2.0
n=004 --- 2.0
n=005 --- 4.0
n=006 --- 6.0
n=007 --- 9.0
n=008 --- 1.5e+01
n=009 --- 2.4e+01
n=010 --- 3.8e+01
n=011 --- 6.1e+01
n=012 --- Inf
n=013 --- Inf
Wyniki dla metody Müllera:

n=000 --- 0.0
n=001 --- 1.0
n=002 --- 1.0
n=003 --- 2.0
n=004 --- 4.0
n=005 --- 8.0
n=006 --- 1.5e+01
n=007 --- 2.8e+01
n=008 --- 5.2e+01
n=009 --- Inf
n=010 --- Inf


In [12]:
f3(x) = x*(2.0^x)-2
df3(x) = 2.0^x * (x * log(2.0)+1.0)
f3_zero = BigFloat(1)
x0_3 = BigFloat(0.123)
@assert(f3(f3_zero) == 0.0, f3(f3_zero))
test_methods(f3, df3, x0_3, f3_zero, 20)

Wyniki dla metody Newtona:

n=000 --- 0.0
n=001 --- 0.0
n=002 --- 0.0
n=003 --- 1.0
n=004 --- 3.0
n=005 --- 7.0
n=006 --- 1.5e+01
n=007 --- 3.0e+01
n=008 --- 4.7e+01
n=009 --- 6.4e+01
n=010 --- Inf
Wyniki dla metody siecznych:

n=000 --- 0.0
n=001 --- 0.0
n=002 --- 0.0
n=003 --- 0.0
n=004 --- 1.0
n=005 --- 2.0
n=006 --- 4.0
n=007 --- 7.0
n=008 --- 1.2e+01
n=009 --- 2.0e+01
n=010 --- 3.3e+01
n=011 --- 5.4e+01
n=012 --- Inf
n=013 --- Inf
Wyniki dla metody Müllera:

n=000 --- 0.0
n=001 --- 0.0
n=002 --- 0.0
n=003 --- 1.0
n=004 --- 2.0
n=005 --- 5.0
n=006 --- 1.0e+01
n=007 --- 2.0e+01
n=008 --- 3.7e+01
n=009 --- 6.9e+01
n=010 --- Inf
n=011 --- Inf


In [13]:
f4(x) = log(abs(x))
df4(x) = 1.0/x
f4_zero = 1.0
x0_4 = BigFloat(0.5)
@assert(f4(f4_zero) == 0.0, f4(f4_zero))
test_methods(f4, df4, x0_4, f4_zero, 20)

Wyniki dla metody Newtona:

n=000 --- 0.0
n=001 --- 0.0
n=002 --- 1.0
n=003 --- 4.0
n=004 --- 8.0
n=005 --- 1.7e+01
n=006 --- 3.4e+01
n=007 --- 7.0e+01
n=008 --- Inf
Wyniki dla metody siecznych:

n=000 --- 0.0
n=001 --- 0.0
n=002 --- 1.0
n=003 --- 2.0
n=004 --- 4.0
n=005 --- 6.0
n=006 --- 1.1e+01
n=007 --- 1.8e+01
n=008 --- 3.0e+01
n=009 --- 4.8e+01
n=010 --- Inf
n=011 --- Inf
Wyniki dla metody Müllera:

n=000 --- 0.0
n=001 --- 0.0
n=002 --- 1.0
n=003 --- 2.0
n=004 --- 5.0
n=005 --- 9.0
n=006 --- 1.8e+01
n=007 --- 3.3e+01
n=008 --- 6.1e+01
n=009 --- Inf
n=010 --- Inf


In [14]:
f5(x) = cos(x*sin(x))
df5(x) = sin(x*sin(x))*(-(sin(x)+x*cos(x)))
f5_zero = BigFloat(-π)/2
x0_5 = BigFloat(-1)
# @assert(f5(f5_zero) == 0.0, f5(f5_zero))
test_methods(f5, df5, x0_5, f5_zero, 20)

Wyniki dla metody Newtona:

n=000 --- 0.0
n=001 --- 1.0
n=002 --- 2.0
n=003 --- 4.0
n=004 --- 9.0
n=005 --- 1.6e+01
n=006 --- 1.6e+01
n=007 --- 1.6e+01
n=008 --- 1.6e+01
Wyniki dla metody siecznych:

n=000 --- 0.0
n=001 --- 1.0
n=002 --- 2.0
n=003 --- 3.0
n=004 --- 5.0
n=005 --- 8.0
n=006 --- 1.3e+01
n=007 --- 1.6e+01
n=008 --- 1.6e+01
n=009 --- 1.6e+01
n=010 --- 1.6e+01
n=011 --- 1.6e+01
Wyniki dla metody Müllera:

n=000 --- 0.0
n=001 --- 1.0
n=002 --- 2.0
n=003 --- 3.0
n=004 --- 6.0
n=005 --- 1.2e+01
n=006 --- 1.6e+01
n=007 --- 1.6e+01
n=008 --- 1.6e+01
n=009 --- 1.6e+01
n=010 --- 1.6e+01
